<a href="https://colab.research.google.com/github/dojian/MLproject/blob/main/graph_nn_points.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
"""
Created on Nov 26 2023

@author: dojian
"""

In [ ]:
# Import libraries

import os
import cv2
import json
import tensorflow as tf
import numpy as np
import pickle

In [28]:
class Data_Loader():

    def __init__(self, data_file_path, label_file_path, n_images, chart_type):
        self.data_file_path = data_file_path
        self.label_file_path = label_file_path
        self.n_images = n_images
        self.chart_type = chart_type  # Added chart_type as an attribute

    def load_image_data(self):
        X = []
        Y = []

        for file in os.listdir(self.data_file_path):
            file_path = os.path.join(self.data_file_path, file)

            if file.endswith('.jpg'):
                img_annotations = self.__load_annotations(file)

                # Check if annotations exist and if chart-type matches self.chart_type
                if img_annotations is not None and img_annotations.get('chart-type') == self.chart_type:
                    Y.append(img_annotations)
                    img = cv2.imread(file_path)
                    X.append(img)

                # Stop if enough images are loaded
                if len(X) >= self.n_images:
                    return X, Y

        return X, Y

    def __load_annotations(self, image_file_name):
        file_name = image_file_name.split('.jpg')[0]
        json_file_name = file_name + '.json'
        json_file_path = os.path.join(self.label_file_path, json_file_name)

        if os.path.isfile(json_file_path):
            with open(json_file_path) as f:
                return json.load(f)

        else:
            return None

In [29]:
class Image_Processor():

    def __init__(self, images):
        self.images = images
        self.min_width, self.min_height = self.__find_smallest_image_width_and_height()
        self.resized_images = self.__resize_images()


    def __resize_images(self):

        X_resized = []

        for img in self.images:
            X_resized.append(tf.image.resize(img,
                                             size=(self.min_width, self.min_height)))

        return np.array(X_resized)


    def __find_smallest_image_width_and_height(self):

        min_width = np.size(self.images[0], 0)
        min_height = np.size(self.images[0], 1)

        for img in self.images[1:]:

            if np.size(img, 0) < min_width:
                min_width = np.size(img, 0)

            if np.size(img, 1) < min_height:
                min_height = np.size(img, 1)

        return min_width, min_height

In [30]:
import os
os.getcwd()

'/content'

In [31]:
# mount your Google drive in Colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
#Load data from Kaggle
!pip install -U -q kaggle
!mkdir -p ~/.kaggle
!echo '{"username":"angel80423","key":"f5a347582d10b1f9f45bc7bb61ab390b"}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c benetech-making-graphs-accessible

benetech-making-graphs-accessible.zip: Skipping, found more recently modified local copy (use --force to force download)


In [33]:
!unzip -q benetech-making-graphs-accessible.zip -d ./

replace ./sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [97]:
data_file_path = '/content/train/images'
label_file_path = '/content/train/annotations'

In [98]:
n_images = 7000
chart_type='scatter'

# Load in raw data
X_raw, Y_raw = Data_Loader(data_file_path,
                           label_file_path, n_images,chart_type).load_image_data()

In [99]:
# Process images: resize and rescale
X_processed = Image_Processor(X_raw)
X_resized = X_processed.resized_images
X_scaled = X_resized / 255.0

In [100]:
print(X_scaled.shape)
print(len(Y_raw))

(7000, 223, 318, 3)
7000


In [101]:
#Process labels: make scatters saved as an array of (x,y) points

def process_labels(Y_raw):
    Y_processed = []
    for annotation in Y_raw:
        # Extract (x,y) for each point from annotation
        data_series = annotation['data-series']
        points = [(point['x'], point['y']) for point in data_series]
        Y_processed.append(points)

    return np.array(Y_processed)

Y_processed =process_labels(Y_raw)

<ipython-input-101-6886b0e1c1f2>:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(Y_processed)


In [102]:
# Find the maximum length
max_length = max(len(item) for item in Y_processed)
# Pad each sublist in Y_processed
Y_padded = np.array([np.pad(item, ((0, max_length - len(item)), (0, 0)), mode='constant') for item in Y_processed])

In [103]:
# Shuffle and Split the Data
shuffle_indices = tf.random.shuffle(tf.range(tf.shape(X_scaled)[0], dtype=tf.int32))
X_shuffled = tf.gather(X_scaled, shuffle_indices)
Y_shuffled = tf.gather(Y_padded, shuffle_indices)

split_index = int(0.8 * len(X_shuffled))

X_train, X_val = X_shuffled[:split_index], X_shuffled[split_index:]
Y_train, Y_val = Y_shuffled[:split_index], Y_shuffled[split_index:]



In [104]:
print("X_train shape:", X_train.shape)
print("X_val shape:", X_val.shape)
print("Y_train shape:", Y_train.shape)
print("Y_val shape:", Y_val.shape)

X_train shape: (5600, 223, 318, 3)
X_val shape: (1400, 223, 318, 3)
Y_train shape: (5600, 100, 2)
Y_val shape: (1400, 100, 2)


In [110]:
def build_model(input_shape):

    model = tf.keras.Sequential()

    # add first convolution layer to the model
    model.add(tf.keras.layers.Conv2D(
        filters=32,
        kernel_size=(5, 5),
        strides=(1, 1),
        padding='same',
        data_format='channels_last',
        name='conv_1',
        activation='relu'))


    # add a max pooling layer with pool size (2,2) and strides of 2
    # (this will reduce the spatial dimensions by half)
    model.add(tf.keras.layers.MaxPool2D(
        pool_size=(2, 2),
        name='pool_1'))


    # add second convolutional layer
    # model.add(tf.keras.layers.Conv2D(
    #     filters=64,
    #     kernel_size=(5, 5),
    #     strides=(1, 1),
    #     padding='same',
    #     name='conv_2',
    #     activation='relu'))

    # # add second max pooling layer with pool size (2,2) and strides of 2
    # # (this will further reduce the spatial dimensions by half)
    # model.add(tf.keras.layers.MaxPool2D(
    #     pool_size=(2, 2), name='pool_2')
    # )


    ## add a fully connected layer (need to flatten the output of the previous layers first)
    model.add(tf.keras.layers.Flatten())
    #model.add(tf.keras.layers.Dense(
    #    units=16,
     #   name='fc_1',
     #   activation='relu'))

    # add dropout layer
    #model.add(tf.keras.layers.Dropout(
    #     rate=0.5))

    # add the last fully connected layer
    # this last layer sets the activation function to "None" in order to output the logits
    model.add(tf.keras.layers.Dense(
        units=max_length*2,
        name='fc_1',
        activation=None))

    # Reshape the output to (batch_size, 70, 2)
    model.add(tf.keras.layers.Reshape((100, 2)))

    tf.random.set_seed(1)
    model.build(input_shape=input_shape)
    model.summary()

    return model

In [ ]:
model = build_model(
     input_shape=(None, X_processed.min_width, X_processed.min_height, 3))

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
               loss=tf.keras.losses.MeanSquaredError(),
               metrics=['mae'])

tf.random.set_seed(1234)
np.random.seed(1234)
history = model.fit(X_train, Y_train,
                     epochs=20,
                     batch_size=32,
                     validation_data=(X_val, Y_val) )

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_1 (Conv2D)             (None, 223, 318, 32)      2432      
                                                                 
 pool_1 (MaxPooling2D)       (None, 111, 159, 32)      0         
                                                                 
 flatten_20 (Flatten)        (None, 564768)            0         
                                                                 
 fc_1 (Dense)                (None, 200)               112953800 
                                                                 
 reshape_5 (Reshape)         (None, 100, 2)            0         
                                                                 
Total params: 112956232 (430.89 MB)
Trainable params: 112956232 (430.89 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1

In [ ]:
!nvidia-smi